In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
import datetime
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants
from shared.normalization import *
import random
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-11 15:15:35.396097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 15:15:35.903420: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [8]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [9]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_0_to_1)

In [10]:
train_data.data

<xarray.DataArray 'data' (participant: 15, epochs: 200, labels: 4,
                          channels: 30, samples: 154)>
array([[[[[       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          ...,
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan]],

         [[0.4633308 , 0.47666781, 0.4880833 , ...,        nan,
                  nan,        nan],
          [0.46270677, 0.48113741, 0.50838311, ...,        nan,
                  nan,        nan],
          [0.46154676, 0.47153108, 0.48150728, ...,        nan,
                  nan,        nan],
...
          [0.41665341, 0.42520935, 0.43346295, ...,        nan,
                  nan,        nan],
          [0.4451984 , 0.43866406, 0.45560413, ...,        nan,
                  nan,        nan],
          [0.48462318, 0.44832508, 0.43694288, ...,        nan,
                  nan,        nan]],

         [[0.43122009, 0.44561045, 0.42866633, ...,        nan,
                  nan,        nan],
          [0.48191239, 0.31326798, 0.37990133, ...,        nan,
                  nan,        nan],
          [0.47560312, 0.41375382, 0.40599513, ...,        nan,
                  nan,        nan],
          ...,
          [0.49405895, 0.52651192, 0.53673716, ...,        nan,
                  nan,        nan],
          [0.53290621, 0.56844441, 0.57092554, ...,        nan,
                  nan,        nan],
          [0.49951581, 0.53447491, 0.55196029, ...,        nan,
                  nan,        nan]]]]])
Coordinates:
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 147 148 149 150 151 152 153
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * participant  (participant) object '0002' '0006' '0007' ... '0022' '0023'
  * labels       (labels) object 'confirmation' 'decision' 'encoding' 'response'
    stim         (participant, epochs) float64 2.0 2.0 1.0 2.0 ... 2.0 1.0 2.0
    resp         (participant, epochs) object 'resp_right' ... 'resp_right'
    rt           (participant, epochs) float64 1.311 1.138 0.934 ... 0.386 1.277
    cue          (participant, epochs) object 'SP' 'SP' 'AC' ... 'SP' 'SP' 'AC'
    movement     (participant, epochs) object 'stim_right' ... 'stim_right'
    trigger      (participant, epochs) object 'SP/stim_right/resp_right' ... ...

In [11]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [12]:
train_and_evaluate(model, train_data, val_data, test_data, epochs=20)

Epoch 1/20
595/595 [==============================] - 11s 16ms/step - loss: 1.3652 - accuracy: 0.2806 - val_loss: 1.3571 - val_accuracy: 0.2861
Epoch 2/20
595/595 [==============================] - 10s 16ms/step - loss: 1.3560 - accuracy: 0.2839 - val_loss: 1.3520 - val_accuracy: 0.2861
Epoch 3/20
595/595 [==============================] - 10s 16ms/step - loss: 1.3546 - accuracy: 0.2913 - val_loss: 1.3521 - val_accuracy: 0.2861
Epoch 4/20
595/595 [==============================] - 10s 16ms/step - loss: 1.3544 - accuracy: 0.2813 - val_loss: 1.3517 - val_accuracy: 0.2861
Epoch 5/20
595/595 [==============================] - 10s 16ms/step - loss: 1.3540 - accuracy: 0.2870 - val_loss: 1.3517 - val_accuracy: 0.2861
Epoch 6/20
195/195 [==============================] - 2s 11ms/step


/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [7]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 147, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 143, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 71, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 69, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 34, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 32, 256)       98560 